In [103]:
import requests
import bs4
from bs4 import BeautifulSoup
import time
import numpy as np
import pandas as pd
import time

In [99]:
from selenium import webdriver  
from selenium.common.exceptions import NoSuchElementException  
from selenium.webdriver.common.keys import Keys  
from bs4 import BeautifulSoup

def get_pop_from_google(city):
    
    city = city.replace(" ", "+")
    URL = "https://www.google.com/search?q={}+population"
    URL = URL.format(city)
    browser = webdriver.Chrome()  
    browser.get(URL)  
    html_source = browser.page_source  
    browser.quit()

    soup = BeautifulSoup(html_source,'html.parser')  
    #class "postText" is not defined in the source code
    value = soup.find('div',{'class':'kpd-ans kno-fb-ctx _fbh'})
    
    if value != None:
        return value.get_text()
    value = soup.find('div',{'class':'_XWk'})
    if value != None:
        return value.get_text()
    return "OOPS"

In [102]:
df_cost_living = pd.read_csv("~/Workspace/Data/cost_of_living.csv")
df_cost_living['search_location'] = df_cost_living.city

In [104]:
population = []
for city in df_cost_living.city:
    population.append(get_pop_from_google(city))
    time.sleep(.1)


In [26]:
def find_city(result):
    loc = result.find(class_="city-name").get_text().split(" (") #getting locatoin and splitting it from county
    city = loc[0]
    country = loc[1][0:-1] #-2 is to get rid of the trailing parenthesis
    price = result.find(class_="price-index").get_text()
    return (city, country, price)

In [109]:
pop = [p.split('\xa0') for p in population]


In [118]:
def format_pop(p):
    # splitting on white space to just get the number in the beginning
    num = p[0].split(' ')[0]
    # getting rid of commas in the string
    num = num.replace(",","")
    # converting num to float
    num = float(num)
    # if there is more info to format with
    if len(p)>1:
        if p[1].startswith('million'):
            num = num * 1000000
    return num 


In [120]:
formatted_pop = [format_pop(p) for p in pop]

In [121]:
df_cost_living['population'] = formatted_pop

In [123]:
df_cost_living.head()
del df_cost_living['city']

In [124]:
df_cost_living.to_csv("~/Workspace/Data/cost_of_living_pop.csv", index=False)